## Child Test

How do you get a test case running that turns out to be too big? 

> Write a smaller
test case that represents the broken part of the bigger test case.

## Mock Object

How do you test an object that relies on another complicated resource?

> Create a fake version of the complicated resource.

```python
def testOrderLookup():
    db = MockDatabase()
    # Add some test data to the database
```

## Self Shunt

Let’s suppose that we are writing a point of sale system. When a sales clerk swipes an item past a barcode **scanner**, its name and price come up on 
an LCD **display**.

We may end up writing test like this:

```python
def testScanner():
    scanner = Scanner()
    display = MockDisplay()
    
    item = scanner.scan()
    display.display(item)
```

But, what do we assert? Do we add a method to the display to ask it whether it has 
displayed anything?

Worse, the test case is acting as an intermediary between the scanner and the 
display.

Let's try a better version:

> [!NOTE]
> If your test acts as a mediator between two objects,
pick one object and let it talk to the other.

```python
def testScanner():
    display = MockDisplay()
    scanner = Scanner(display)
    
    item = Item('Cornflakes')
    scanner.scan(item)  # call the display.displayItem method

    assert item == display.lastItem


class MockDisplay:

    lastItem = None

    def displayItem(self, item):
        self.lastItem = item
```

Pros: the test is simple and easy to read.

Cons: The MockDisplay is untested code. I have seen a tendency for behavior to slip into, which is disastorous. It can cause a bug in your tests that take too long to track down.

> [!NOTE]
> Let's say that you are a test case. One of the things that you can do is pass 
yourself to the objects you are testing.

```python
class InterfaceDisplay(ABC):

    @abstractmethod
    def displayItem(self, item):
        pass


class testScanner(InterfaceDisplay):

    lastItem = None

    def displayItem(self, item):
        self.lastItem = item

    def testScanner(self):
        scanner = Scanner(self) # pass the test case to the scanner
        item = Item('Cornflakes')
        scanner.scan(item)  # call the "self" displayItem method

        assert item == self.lastItem
```

> [!NOTE]
> How do you test that one object communicates correctly with another? Have
the object under test communicate with the test case instead of with the object
it expects.

The implementation on the interface in the test class will give you a good idea on what the implementation of the concrete class should look like when you get there.

## Log String

A common problem is that the test method may not return values. In this case, you want to test if the methods were called and called in the correct order.

When you call a method, you create a log entry to prove that the method was called.

```python
def testFoo():
    net = Net()
    img = np.random.rand(256, 256, 3)

    net.predict(img)

    assert net.called_methods == ['predict', 'preprocess']  # expect that the preprocess must be called
    assert net.messages == ['shape=(256, 256, 3)', 'shape=(224, 224, 3)']


class Net:

    messages = []
    called_methods = []
    model = ResNet50()

    def predict(self, img):
        self.called_methods.append('predict')
        self.messages.append('shape={}'.format(img.shape))
        
        img = self.preprocess(img)

        return self.model(img)


    def preprocess(self, img):
        self.called_methods.append('preprocess')
        self.messages.append('shape={}'.format(img.shape))

        img = cv2.resize(img, (224, 224))

        return img
```

Log String works well with Self Shunt. The test case implements the methods in the shunted interface by adding to the log and then returning reasonable
values.

## Crash Test Dummy

Let's say you want to test what happends to our application when the file system is full. 

You could create a very big file and fill the system. Doing this is a bad idea.

We'll simulate it.

In [9]:
def open_full_file(path):

    class FullFile:
        
        def read(self):
            raise IOError()
        
    return FullFile()


def testFileSystemError():
    file = open_full_file('foo.txt')
    try:
        file.read()
        assert False
    except IOError:
        assert True

    
testFileSystemError()

## References

1. https://citeseerx.ist.psu.edu/document?repid=rep1&type=pdf&doi=3e1876a233963013d84bcba6a66289d623c63d57

2. https://8thlight.com/insights/self-shunt